In [1]:
import json

import boto3
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [2]:
secretsmanager = {
    "region": "eu-west-1",
    "mlflow_secret": "acme/mlflow"
}

session = boto3.session.Session()
client = session.client(service_name="secretsmanager", region_name=secretsmanager["region"])
mlflow_secret = client.get_secret_value(SecretId=secretsmanager["mlflow_secret"])
mlflowdb_conf = json.loads(mlflow_secret["SecretString"])
tracking_uri = f"mysql+pymysql://{mlflowdb_conf['username']}:{mlflowdb_conf['password']}@{mlflowdb_conf['host']}/mlflow"

In [3]:
mlflow_client = MlflowClient(tracking_uri)

In [32]:
EXPERIMENTS_TO_CLEANUP = [
    "cifar10-test"
]

In [33]:
for expt_name in EXPERIMENTS_TO_CLEANUP:
    expt = mlflow_client.get_experiment_by_name(expt_name)
    runs = [run_info for run_info in mlflow_client.list_run_infos(expt.experiment_id, run_view_type=ViewType.DELETED_ONLY)]
    print(f"Found {len(runs)} deleted runs in experiment: {expt_name}. Deleting...")
    
    for run_info in runs:
        artifact_uri_split = run_info.artifact_uri.split('/')
        bucket_name = artifact_uri_split[2]
        prefix = '/'.join(artifact_uri_split[3:artifact_uri_split.index(run_info.run_id) + 1])
        
        print(f"Artifacts URI: {run_info.artifact_uri}. Deleting...")

        s3 = boto3.resource('s3')
        bucket = s3.Bucket(bucket_name)
        bucket.objects.filter(Prefix=prefix).delete()

Found 0 deleted runs in experiment: cifar10-test. Deleting...
